<a href="https://colab.research.google.com/github/AkshathaBolla/ANN/blob/main/CyberForce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module I: Practice Exercise - 'Cyber-Force AI' (Advanced Spec)

### **Context: Network Intrusion Detection**
You are the Lead Security Engineer for 'Cyber-Force'. You have traffic logs from 500 devices.
Your sensors measure:
1.  **Packet Size (Bytes)**
2.  **Request Latency (ms)**
3.  **Encryption Entropy (0-10)**

You have **two separate goals**:
* **Goal A (Severity):** Predict the **Threat Severity Score (0-100)**. (Regression)
* **Goal B (Attack Type):** Classify the traffic: **0=Benign, 1=DDoS, 2=Phishing**. (Multi-Class Classification)

---
**INSTRUCTIONS:**
This exercise uses **Early Stopping** and **L2 Regularization** to handle overfitting.
The architectures are **Deeper (3+ layers)** and **Wider (128+ neurons)**.

In [1]:
# CELL 1: DATA GENERATION (Run this first)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(999)

# 500 Traffic Samples
N = 500
# Features: Size (64-1500), Latency (10-500), Entropy (0-10)
X = torch.rand(N, 3) * torch.tensor([1436, 490, 10]) + torch.tensor([64, 10, 0])

# Target A: Severity Score (Regression)
# Severity increases with Entropy and Latency
y_sev = (X[:, 1] * 0.1) + (X[:, 2] * 5) + torch.randn(N) * 5
y_sev = y_sev.view(-1, 1).float()

# Target B: Attack Type (Multi-Class 0, 1, 2)
# Logic: High Latency -> DDoS (1), High Entropy -> Phishing (2), Else Benign (0)
y_type = torch.zeros(N).long()
y_type[X[:, 1] > 300] = 1 # DDoS
y_type[X[:, 2] > 8] = 2   # Phishing

print(f"Data Ready. X: {X.shape}, y_sev: {y_sev.shape}, y_type: {y_type.shape}")

Data Ready. X: torch.Size([500, 3]), y_sev: torch.Size([500, 1]), y_type: torch.Size([500])


## **Level 1: Severity Prediction (Deep Regression)**
**Your Task:** Predict Threat Severity.

**Blueprint (Architecture):**
1.  **Input:** 3 Features
2.  **Layer 1:** 64 Neurons, `ReLU`
3.  **Layer 2:** 32 Neurons, `ReLU`
4.  **Layer 3:** 16 Neurons, `ReLU`
5.  **Output:** 1 Neuron (Linear)

**Training Specs:**
* Loss: `MSELoss`
* Optimizer: `Adam` (lr=0.005)
* Epochs: 150

In [2]:
# LEVEL 1: WRITE YOUR CODE HERE

# 1. Define 'SeverityModel' (3 Hidden Layers!)
class SeverityModel(nn.Module):
    # TODO: Linear(3,64) -> ReLU -> Linear(64,32) -> ReLU -> Linear(32,16) -> ReLU -> Linear(16,1)
  def __init__(self):
    super(SeverityModel,self).__init__()
    self.layer1=nn.Linear(3,64)
    self.layer2=nn.ReLU()
    self.layer3=nn.Linear(64,32)
    self.layer4=nn.ReLU()
    self.layer5=nn.Linear(32,16)
    self.layer6=nn.ReLU()
    self.layer7=nn.Linear(16,1)
  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
    x=self.layer5(x)
    x=self.layer6(x)
    x=self.layer7(x)
    return x
model_sev = SeverityModel()

# 2. Optimizer & Loss
criterion=nn.MSELoss()
optimizer=optim.Adam(model_sev.parameters(),lr=0.005)

# 3. Training Loop
for epoch in range(150):
  optimizer.zero_grad()
  outputs=model_sev(X)
  loss=criterion(outputs,y_sev)
  loss.backward()
  optimizer.step()
  if (epoch+1)%10==0:
    print(f'Epoch [{epoch+1}/150], Loss: {loss.item():.4f}')
# 4. Check Loss

Epoch [10/150], Loss: 379.1077
Epoch [20/150], Loss: 320.8535
Epoch [30/150], Loss: 300.6827
Epoch [40/150], Loss: 298.0529
Epoch [50/150], Loss: 293.9697
Epoch [60/150], Loss: 289.8494
Epoch [70/150], Loss: 286.1889
Epoch [80/150], Loss: 282.4418
Epoch [90/150], Loss: 278.1497
Epoch [100/150], Loss: 272.9839
Epoch [110/150], Loss: 266.5392
Epoch [120/150], Loss: 257.9740
Epoch [130/150], Loss: 245.7299
Epoch [140/150], Loss: 224.3733
Epoch [150/150], Loss: 214.6802


In [3]:
# TEST LEVEL 1
try:
    # Check Depth
    layers = [m for m in model_sev.modules() if isinstance(m, nn.Linear)]
    assert len(layers) == 4, f"Blueprint requires 4 Linear layers. Found {len(layers)}"
    assert layers[0].out_features == 64
    assert layers[1].out_features == 32

    print("✅ Level 1 Passed: Deep Regression Architecture Correct.")
except Exception as e:
    print(f"❌ Level 1 Fail: {e}")

✅ Level 1 Passed: Deep Regression Architecture Correct.


## **Level 2: Attack Classification (Overfitting & Early Stopping)**
**Your Task:** Predict Attack Type (0, 1, 2).

**Step 2.1: Pipeline**
* Split: **50 Train / 450 Test** (Tiny training set to force overfitting!).
* `DataLoader` (Batch=10).

**Step 2.2: The 'Overkill' Model**
* `Linear(3, 128) -> ReLU`
* `Linear(128, 128) -> ReLU`
* `Linear(128, 3)`

**Step 2.3: Diagnosis**
* Train for 200 epochs. Observe that Test Loss goes UP while Train Loss goes DOWN.

**Step 2.4: The Fix (Early Stopping & L2 Regularization)**
* **Fix 1:** Add `weight_decay=0.01` to the Adam optimizer (L2 Regularization).
* **Fix 2:** Implement **Early Stopping** inside the loop:
    * If `test_loss` does not improve for 10 epochs (patience), `break` the loop.
    * *Hint: Keep track of `best_loss`.*

In [4]:
# LEVEL 2: WRITE YOUR CODE HERE

# 1. Pipeline (50 Train / 450 Test)
X_train,X_test=X[:50],X[50:]
y_train,y_test=y_type[:50],y_type[50:]

train_ds=TensorDataset(X_train,y_train)
test_ds=TensorDataset(X_test,y_test)

train_loader=DataLoader(train_ds,batch_size=10,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=10,shuffle=False)
# 2. Define 'model_overkill'

class Modeloverkill(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(3,128)
    self.relu=nn.ReLU()
    self.linear2=nn.Linear(128,128)
    self.relu2=nn.ReLU()
    self.linear3=nn.Linear(128,3)

  def forward(self,x):
    x=self.linear(x)
    x=self.relu(x)
    x=self.linear2(x)
    x=self.relu2(x)
    x=self.linear3(x)
    return x

model_overkill=Modeloverkill()
# 3. Optimizer WITH L2 Regularization (weight_decay=0.01)
opt_l2 = optim.Adam(model_overkill.parameters(), lr=0.001, weight_decay=0.01)

# 4. Training Loop WITH Early Stopping
best_loss = float('inf')
patience = 10
trigger_times = 0
criterion=nn.CrossEntropyLoss()

for epoch in range(200):
#     Train...
      model_overkill.train()
      for inputs, targets in train_loader:
          opt_l2.zero_grad()
          outputs = model_overkill(inputs)
          loss = criterion(outputs, targets)
          loss.backward()
          opt_l2.step()
#     Calc Test Loss...
      model_overkill.eval()
      with torch.no_grad():
          for inputs, targets in test_loader:
              outputs = model_overkill(inputs)
              test_loss = criterion(outputs, targets).item()
#     Early Stopping Logic:
      if test_loss < best_loss:
             best_loss = test_loss
             trigger_times = 0
      else:
             trigger_times += 1
             if trigger_times >= patience:
                 print("Early Stopping!")
                 break

Early Stopping!


In [5]:
# TEST LEVEL 2
try:
    # Check L2
    assert opt_l2.defaults['weight_decay'] == 0.01, "Optimizer must have weight_decay=0.01"

    # Check Width
    l1 = list(model_overkill.modules())[1]
    if isinstance(l1, nn.Sequential): l1 = l1[0]
    assert l1.out_features == 128, "Hidden layer must be 128 wide"

    print("✅ Level 2 Passed: Early Stopping & L2 Logic Valid.")
except Exception as e:
    print(f"❌ Level 2 Fail: {e}")

✅ Level 2 Passed: Early Stopping & L2 Logic Valid.


## **Level 3: The Tournament (Activations)**

**Part 3.1: Manual CrossEntropy (Concept)**
Remember: `CrossEntropyLoss` combines `LogSoftmax` and `NLLLoss`.

**Part 3.2: The Tournament**
Compare 3 configs on **Attack Classification**:

1.  **"Baseline"**: `ReLU` + `Adam` (Standard)
2.  **"Dying ReLU Fix"**: `LeakyReLU` + `SGD`
3.  **"Smooth"**: `ELU` + `Adam`
    * *Note: ELU (Exponential Linear Unit) can often converge faster than ReLU.*

In [10]:
# LEVEL 3: TOURNAMENT
experiments = [
    {"name": "Baseline", "act": nn.ReLU(), "opt": optim.Adam}, # ReLU, Adam
    {"name": "Leaky",    "act": nn.LeakyReLU(), "opt":optim.SGD}, # LeakyReLU, SGD
    {"name": "Smooth",   "act": nn.ELU(), "opt": optim.Adam}  # ELU, Adam
]

print(f"{'Name':<10} | {'Test Acc':<10}")
print("-"*25)

# Loop...

Name       | Test Acc  
-------------------------


In [11]:
# TEST LEVEL 3
try:
    assert isinstance(experiments[1]['act'], nn.LeakyReLU), "Exp 2 Act must be LeakyReLU"
    assert isinstance(experiments[2]['act'], nn.ELU), "Exp 3 Act must be ELU"

    print("✅ Level 3 Passed: Advanced Activations Configured.")
except Exception as e:
    print(f"❌ Level 3 Fail: {e}")

✅ Level 3 Passed: Advanced Activations Configured.


## **Level 4: The Mechanic (2-Input Gradient Descent)**
**Your Task:** Train a model manually (No `optim` library).

**Goal:** Learn $y = 0.5x_1 - 2.0x_2 + 10$

1.  **Data:** $X$ has shape `(N, 2)`.
2.  **Model:** `nn.Linear(2, 1)`.
3.  **Update Rule:** Manual Gradient Descent.
4.  **Important:** Use `with torch.no_grad():`.

In [12]:
# LEVEL 4: THE PURGE
import torch.optim as optim
import gc
for var in list(locals().keys()):
    if 'opt' in var or 'optimizer' in var:
        del locals()[var]
gc.collect()
print("Optimizers deleted. You are on your own.")

Optimizers deleted. You are on your own.


In [22]:
# LEVEL 4: WRITE YOUR CODE HERE

# 1. Setup Data (2 Inputs)
X_vec = torch.tensor([[1.0, 1.0], [2.0, 4.0], [5.0, 1.0]])
#Target: 0.5*x1 - 2.0*x2 + 10
y_vec = (0.5 * X_vec[:, 0] - 2.0 * X_vec[:, 1] + 10).view(-1, 1)

# 2. Define Model
model_vec = nn.Linear(2, 1)

lr=0.01
criterion=nn.MSELoss()
# 3. Manual Loop
for epoch in range(5000):
  #Forward
  preds=model_vec(X_vec)
  loss=criterion(preds,y_vec)
  #backward
  loss.backward()

    # Update
  with torch.no_grad():
       model_vec.weight-=lr*model_vec.weight.grad
       model_vec.bias-=lr*model_vec.bias.grad




    # Zero grad
  model_vec.weight.grad.zero_()
  model_vec.bias.grad.zero_()


In [23]:
# TEST LEVEL 4
try:
    w = model_vec.weight.data[0]
    b = model_vec.bias.item()

    print(f"Final Weights: {w}")
    print(f"Final Bias: {b}")

    assert abs(w[0] - 0.5) < 0.2, f"w1 should be ~0.5"
    assert abs(w[1] + 2.0) < 0.2, f"w2 should be ~ -2.0"
    assert abs(b - 10.0) < 0.2, f"Bias should be ~10.0"

    print("✅ Level 4 Passed: 2-Input Gradient Descent successful!")
except Exception as e:
    print(f"❌ Level 4 Fail: {e}")

Final Weights: tensor([ 0.5000, -2.0000])
Final Bias: 9.999824523925781
✅ Level 4 Passed: 2-Input Gradient Descent successful!
